In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=RuntimeWarning)
warnings.simplefilter("ignore")

In [ ]:
from IPython.display import YouTubeVideo, Image

# [XARRAY](https://github.com/xray/xray)

[xarray](https://github.com/xarray/xarray) (formerly `xray`) has been developed by scientists / engineers working at the [Climate Corporation](http://climate.com/)

It is an open source project and Python package that aims to bring
the labeled data power of [pandas](http://pandas.pydata.org) to the
physical sciences, by providing N-dimensional variants of the core
[pandas](http://pandas.pydata.org) data structures, `Series` and
`DataFrame`: the xarray `DataArray` and `Dataset`.

the goal is to provide a pandas-like and pandas-compatible toolkit for
analytics on multi-dimensional arrays, rather than the tabular data for
which pandas excels. The approach adopts the [Common Data
Model](http://www.unidata.ucar.edu/software/thredds/current/netcdf-java/CDM)
for self-describing scientific data in widespread use in the Earth
sciences (e.g., [netCDF](http://www.unidata.ucar.edu/software/netcdf)
and [OPeNDAP](http://www.opendap.org/)): `xray.Dataset` is an in-memory
representation of a netCDF file.

-   HTML documentation: <http://xarray.readthedocs.org>: **really good doc !**
-   Source code: <http://github.com/xarray/xarray>

The main advantages of using [xarray](https://github.com/xarray/xarray) versus [netCDF4](https://github.com/Unidata/netcdf4-python) are: 

+ intelligent selection along **labelled dimensions** (and also indexes)
+ **groupby** operations
+ **resampling** operations
+ data alignment 
+ IO (netcdf)
+ conversion from / to [Pandas.DataFrames](http://pandas.pydata.org/pandas-docs/dev/generated/pandas.DataFrame.html)


To install the latest version of xarray (via conda): 

    ᐅ conda install xarray

or if you want the bleeding edge: 


    ᐅ pip install git+https://github.com/xarray/xarray

There's too much to see in the context of this talk, to know more about all the cool **xarray** features, watch: 

PyData talk by **Stefan Hoyer**: <https://www.youtube.com/watch?v=T5CZyNwBa9c>

In [ ]:
YouTubeVideo('T5CZyNwBa9c', width=500, height=400, start=0)

## Some examples

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt

In [ ]:
import os
import numpy as np
import pandas as pd

In [ ]:
import xarray as xr; print(xr.__version__)

### Open a netcdf file: monthly HGT from NCEP / NCAR from January 1948 to July 2017

The file (270 Mb) can be downloaded at [ftp://ftp.cdc.noaa.gov/Datasets/ncep.reanalysis.derived/pressure/hgt.mon.mean.nc](ftp://ftp.cdc.noaa.gov/Datasets/ncep.reanalysis.derived/pressure/hgt.mon.mean.nc)

In [ ]:
dset = xr.open_dataset('../data/hgt.2019.nc')

In [ ]:
dset.info

**dset** is a [xray.Dataset](http://xray.readthedocs.org/en/stable/data-structures.html#dataset), It is a dict-like container of labeled arrays (DataArray objects) with aligned dimensions. It is designed as an in-memory representation of the data model from the netCDF file format.

In [ ]:
Image('http://xarray.pydata.org/en/stable/_images/dataset-diagram.png', width=700)

In [ ]:
dset.dims

### accessing variables

In [ ]:
lat = dset['lat']

In [ ]:
lat

In [ ]:
type(lat)

In [ ]:
lat.attrs

In [ ]:
type(lat.data)

In [ ]:
lat.data

### selecting a Dataset along dimensions

In [ ]:
dset

In [ ]:
dset.sel(time='2019-01-01')

### integer indexing also available via `isel`

In [ ]:
dset.isel(time=0)

### and you can slice along one or multiple dimensions 

In [ ]:
dset.sel(time=slice('2019-01-01','2019-01-31'))

### when selecting along any dimension, you need to respect the ORDER of the dimensions, i.e. in this case, the latitudes go from NORTH TO SOUTH 

In [ ]:
dset.sel(time=slice('2019-01-01','2019-01-31'), lat=slice(40,-40))

### or you can sort 

In [ ]:
if dset.lat[0] > dset.lat[-1]: 
    dset = dset.sortby('lat')

In [ ]:
dset.sel(time=slice('2019-01-01','2019-01-31'), lat=slice(-40,40))

In [ ]:
subset = dset.sel(time='2019-01-01', level=1000, lat=slice(-50,40), lon=slice(100, 220))

In [ ]:
subset

In [ ]:
subset = subset.drop('time_bnds')

In [ ]:
subset

In [ ]:
subset['hgt']

In [ ]:
subset['hgt'].plot()

### basic mapping with cartopy

In [ ]:
import cartopy.crs as ccrs

In [ ]:
crs = ccrs.PlateCarree(central_longitude=180)

In [ ]:
f, ax = plt.subplots(figsize=(10,8), subplot_kw={'projection':crs})
subset['hgt'].plot.contourf(ax=ax, transform=ccrs.PlateCarree(), levels=20);
ax.coastlines(resolution='10m')
f.savefig('../figures/map_hgt', dpi=200, bbox_inches='tight')

In [ ]:
crs = ccrs.Orthographic(central_longitude=160)

In [ ]:
f, ax = plt.subplots(subplot_kw={'projection':crs}, figsize=(10, 8))
# dset.isel(time=0, level=0)['hgt'].plot.contourf(ax=ax, transform=ccrs.PlateCarree(central_longitude=0), levels=20);
subset['hgt'].plot.contourf(ax=ax, transform=ccrs.PlateCarree(), levels=20);
ax.set_global()
ax.coastlines(resolution='50m')

### important to close the datasets, or else you might encounter issues 

In [ ]:
dset.close()

In [ ]:
subset.close()

### reading multiple files datasets 

reading datasets that are split in different files (i.e. one file per year or month) is relatively easy in xarray, you just need to pass a LIST (Python list) of file paths

In [ ]:
import pathlib

In [ ]:
path = pathlib.Path('/home/nicolasf/drives/auck_scratch/fauchereaun/Wellington_Python_Workshop_data/')

In [ ]:
lfiles = list(path.rglob("ersst.??????.nc"))

In [ ]:
lfiles.sort()

In [ ]:
len(lfiles)

In [ ]:
lfiles[0]

In [ ]:
lfiles[-1]

In [ ]:
dset = xr.open_mfdataset(lfiles)

In [ ]:
dset

In [ ]:
dset.nbytes / 1e6

In [ ]:
dset = dset.squeeze()

In [ ]:
subset = dset.sel(lon=slice(100., 280.), lat=slice(-5., 5.))

In [ ]:
subset

### aggregation functions along dimensions

you can apply functions along the dimensions of a **xarray** dataset, i.e. calculate a mean over time, or (for zonal means), over latitudes

In [ ]:
time_mean = subset.mean('time')

In [ ]:
time_mean

In [ ]:
time_mean['sst'].plot(cmap=plt.get_cmap('RdBu_r'))

In [ ]:
zonal_mean = subset.mean('lat')

In [ ]:
zonal_mean

In [ ]:
f, my_axes = plt.subplots(figsize=(8,8))
zonal_mean.sel(time=slice('2018', None))['ssta'].plot.contourf(ax=my_axes, levels=30)
my_axes.set_title('ZONAL MEAN SSTs')
my_axes.invert_yaxis()

### And that can be handy to calculate an index ... example with NINO3.4

NINO3.4 is regional avegare of the SST anomalies in the domain (5N-5S, 170W-120W)

In [ ]:
index = subset.sel(lon=slice(360-170, 360-120))

In [ ]:
NINO34 = index.mean(dim=('lon','lat'))

or 

In [ ]:
NINO34 = index.mean('lon').mean('lat')

In [ ]:
NINO34['ssta'].plot()

### calculates a monthly climatology using the groupby machinery

In [ ]:
Image(filename='images/split-apply-combine.png', width=800)

### CLIMATOLOGY

In [ ]:
path

In [ ]:
sfc_temp = xr.open_dataset(path / 'air.mon.mean.nc')

In [ ]:
sfc_temp

In [ ]:
clim = sfc_temp.groupby('time.month').mean('time')

In [ ]:
clim

In [ ]:
clim.sel(month = 8)['air'].plot(cmap=plt.get_cmap('RdBu_r'))

In [ ]:
from calendar import month_abbr

In [ ]:
months = month_abbr[1:]

In [ ]:
months

In [ ]:
clim['month'] = (('month'), months)

In [ ]:
clim

In [ ]:
clim.sel(month = 'Jan')['air'].plot(cmap=plt.get_cmap('RdBu_r'))

### the plot method accepts parameters allowing to fo facetted plots 

In [ ]:
clim['air'].plot(x='lon', y='lat', col='month', col_wrap=4)

### to plot in on a proper map, use cartopy

In [ ]:
from calendar import month_name

In [ ]:
proj = ccrs.PlateCarree(central_longitude=180)

In [ ]:
transform = ccrs.PlateCarree(central_longitude=0)

In [ ]:
months

In [ ]:
f, axes = plt.subplots(nrows=4, ncols=3, figsize=(14,10), subplot_kw={'projection':proj})
f.subplots_adjust(hspace=0.1, wspace=0.1)
axes = axes.flatten()
for i, month in enumerate(months): 
    ax = axes[i]
    clim.sel(month = month)['air'].plot.contourf(ax=ax,transform=transform, cmap=plt.get_cmap('RdBu_r'), levels=np.arange(-50, 55, 5), extend='both')
    ax.coastlines(resolution='50m')

**NOTE**: If you have **DAILY** data, you can calculate a daily climatology using the `dayofyear` attribute, e.g.: 
    
```python 

clim = dset.groupby('time.dayofyear').mean('time')

```

### calculates a seasonal (DJF, MAM, ...) climatology

In [ ]:
seas_clim = sfc_temp.groupby('time.season').mean('time')

In [ ]:
seas_clim

In [ ]:
f, axes = plt.subplots(nrows=2, ncols=2, figsize=(14,10), subplot_kw={'projection':proj})
f.subplots_adjust(hspace=0.1, wspace=0.1)
axes = axes.flatten('F')
for i, seas in enumerate(seas_clim['season'].values): 
    ax = axes[i]
    seas_clim.sel(season = seas)['air'].plot.contourf(ax=ax,transform=transform, cmap=plt.get_cmap('RdBu_r'), levels=np.arange(-50, 55, 5), extend='both')
    ax.coastlines(resolution='50m')

### calculates anomalies with respect to a specific climatological *normal*

#### 1. defines the function

In [ ]:
def demean(x): 
    return x - x.sel(time=slice('1981-01-01','2010-12-01')).mean('time')

#### 2. apply the function to the groupby object

In [ ]:
sfc_anoms = sfc_temp.groupby('time.month').apply(demean) 

In [ ]:
sfc_anoms

In [ ]:
sfc_anoms.isel(time=-1)['air'].plot(cbar_kwargs={'orientation':'horizontal','shrink':0.9, 'label':u'\N{DEGREE SIGN}C'})

In [ ]:
crs = ccrs.PlateCarree(central_longitude=180)

In [ ]:
f, ax = plt.subplots(figsize=(10,8), subplot_kw={'projection':crs})
sfc_anoms.isel(time=-1)['air'].plot.contourf(ax=ax, transform=ccrs.PlateCarree(), levels=20, cbar_kwargs={'shrink':0.5});
ax.coastlines(resolution='50m')

In [ ]:
global_anom = sfc_anoms.mean(dim=('lat','lon'))

In [ ]:
global_anom['air'].plot()

In [ ]:
f, ax = plt.subplots()
global_anom.rolling(dim={'time':12}, min_periods=12, center=True).mean()['air'].plot(ax=ax)
ax.axhline(0, color='k', lw=0.5)

### then you can export it to a dataframe, and e.g. save it into a csv file 

In [ ]:
global_anom_df = global_anom['air'].to_dataframe()

In [ ]:
global_anom_df.head()

### Creates a xray dataset object from numpy arrays

In [ ]:
lon = np.linspace(0, 357.5, 144, endpoint=True)

lat = np.linspace(-90,90, 73, endpoint=True)

lons, lats = np.meshgrid(lon,lat)

lev = np.array([1000,925,850])

time = pd.date_range(start='2015-1-1',end='2015-1-3')

In [ ]:
lat

In [ ]:
arr = np.random.randn(3,3,73,144)

The dictionnary **keys** are the **variables** contained in the Dataset.<br><br>
The Dictionnary **values** are **tuples**, with first the (or the list of) dimension(s) over which the array varies, then the array itself

In [ ]:
d = {}
d['time'] = ('time',time)
d['latitudes'] = ('latitudes',lat)
d['longitudes'] = ('longitudes', lon)
d['level'] = ('level', lev)
d['var'] = (['time','level','latitudes','longitudes'], arr)

In [ ]:
dset = xr.Dataset(d)

In [ ]:
dset

adding global attributes

In [ ]:
dset.attrs['author'] = 'nicolas.fauchereau@niwa.co.nz'

In [ ]:
dset

adding variables attributes

In [ ]:
dset.longitudes.attrs['units'] = 'degrees_east'
dset.latitudes.attrs['units'] = 'degrees_north'

In [ ]:
dset.latitudes.attrs

In [ ]:
dset.sel(time='2015-1-2', level=1000)

In [ ]:
dset.to_netcdf('../data/dset_from_dict.nc')

In [ ]:
!ncdump -h ../data/dset_from_dict.nc

### Creates a xray dataset object from a Pandas DataFrame

In [ ]:
import string
df = pd.DataFrame(np.random.randn(365,5), \
                  index=pd.date_range(start='2014-1-1', periods=365), \
                  columns=list(string.ascii_letters[:5]))

In [ ]:
df.head()

In [ ]:
df_ds = xr.Dataset.from_dataframe(df)

In [ ]:
df_ds

In [ ]:
group = df_ds.groupby('index.month').mean('index')

In [ ]:
group

### Opening a file over the network with openDAP

In [ ]:
url = 'http://www.esrl.noaa.gov/psd/thredds/dodsC/Datasets/interp_OLR/olr.mon.mean.nc'

In [ ]:
olr_dset = xr.open_dataset(url)

In [ ]:
olr_sub = olr_dset.sel(time='1998-1-1',lat=slice(30,-30), lon=slice(170, 300))

In [ ]:
olr_sub

In [ ]:
f, ax = plt.subplots(figsize=(10,8), subplot_kw={'projection':ccrs.PlateCarree(central_longitude=180)})
olr_sub['olr'].plot.contourf(ax=ax, transform=ccrs.PlateCarree(), levels=20, cbar_kwargs={'shrink':0.5});
ax.coastlines(resolution='50m')

### now with the gridlines and lat / lon labels 

In [ ]:
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter

In [ ]:
f, ax = plt.subplots(figsize=(10,8), subplot_kw={'projection':ccrs.PlateCarree(central_longitude=180)})

olr_sub['olr'].plot.contourf(ax=ax, transform=ccrs.PlateCarree(), levels=20, cbar_kwargs={'shrink':0.5});

ax.coastlines(resolution='50m')

xticks = np.arange(170, 300 + 30, 30)

yticks = np.arange(-30., 30., 10.)

gl = ax.gridlines(draw_labels=False, linewidth=0.5, linestyle='--', xlocs=xticks, ylocs=yticks, crs=ccrs.PlateCarree())

ax.set_xticks(xticks, crs=ccrs.PlateCarree())

ax.set_yticks(yticks, crs=ccrs.PlateCarree())

lon_formatter = LongitudeFormatter(zero_direction_label=True, dateline_direction_label=True)

lat_formatter = LatitudeFormatter()

ax.xaxis.set_major_formatter(lon_formatter)

ax.yaxis.set_major_formatter(lat_formatter)

ax.set_extent([170, 300, -30, 30], crs=ccrs.PlateCarree())

ax.set_xlabel('longitude')

ax.set_xlabel('latitude')

f.savefig('../figures/olr_map.png', bbox_inches='tight', dpi=200)

In [ ]:
!eom ../figures/olr_map.png

### interpolation from one grid to another 

In [ ]:
olr_sub

In [ ]:
olr_sub = olr_sub.sortby('lat')

In [ ]:
olr_sub

In [ ]:
d = {}
d['lon'] = (('lon'), np.arange(olr_sub.lon[0], olr_sub.lon[-1] + 0.25, 0.25))
d['lat'] = (('lat'), np.arange(olr_sub.lat[0], olr_sub.lat[-1] + 0.25, 0.25))

In [ ]:
target_grid = xr.Dataset(d)

In [ ]:
target_grid

In [ ]:
olr_sub_interp = olr_sub.interp_like(target_grid)

In [ ]:
olr_sub_interp

In [ ]:
f, axes = plt.subplots(nrows=2, figsize=(10,10))

olr_sub['olr'].plot.imshow(ax=axes[0], interpolation='none')

olr_sub_interp['olr'].plot.imshow(ax=axes[1], interpolation='none')
